# This Code is now integrated to the end of the Scrapper Notebook

## This notebook takes the database created by the raw scrape of game results, cleans up some problems with the data 

### Task List
- the advanced metrics tables should have the team name added to each player's row
    - can also probably store as a single table instead of two tables
        - would also want to add home or away to each row along with Team Name
        Should be able to do it with a rather simple if then and the team names in the Game_ID

- Import the master rosters that are scraped and stored as CSV into the database so we can join data on age, class rank, ect 
    - I 

In [1]:
## Dependencies

import pandas as pd
import numpy as np

import sqlite3

# db_path = '../data/2022-2023 Season Data.db' # Set FOr 2022-2023 Season

# db_path = '../TEMP/2023_Season_Nov 2_Game_Stats.db' # Set For 2023-2024 Season

# db_path = '../data/db/Current_Season_YTD_Game_Stats.db' # Set For 2023-2024 Season

# db_path = '../TEMP/2022_Game_Stats.db' # 2022 Season

# db_path = '../data/db/2021_Season_v1_Game_Stats.db' # 2021 Season

db_path = '../TEMP/2020_Game_Stats.db'

# db_path = '../TEMP/MAR_18_Current_YTD_Stats_Game_Stats.db'



conn = sqlite3.connect(db_path)

# Roster data
folder = '../data/rosters/'

df_2023 = pd.read_csv(folder + '2023_master_roster.csv')
df_2022 = pd.read_csv(folder + '2022_master_roster.csv')
df_2021 = pd.read_csv(folder + '2021_master_roster.csv')
df_2020 = pd.read_csv(folder + '2020_master_roster.csv')


# Load the Correct Roster into the database
roster_df = df_2020.copy()

# Set the SeasonYear in the database_roster
season_year_setting = 2020

## Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())




[('game_details',), ('scoring_summary',), ('penalty_summary',), ('goalie_stats',), ('player_stats',), ('line_chart',), ('linescore',), ('advanced_metrics',)]


In [2]:
roster_df.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,NaN,4,"Bengtsson, Jacob",So,6-3,202,5/8/1999,"Stockholm, Sweden",Defensemen,75,NaN,NaN,NaN,Waterloo,USHL,Lake-Superior
1,NaN,23,"Borshyov, Artyom",So,6-3,210,8/22/2000,"Vitebsk, Belarus",Defensemen,75,NaN,NaN,NaN,Youngstown,USHL,Lake-Superior
2,NaN,25,"Gervais, Jeremy",So,5-11,180,1/9/1999,"Prince George, B.C.",Defensemen,71,NaN,NaN,NaN,Salmon Arm,BCHL,Lake-Superior
3,NaN,5,"Henrikson, Arvid",Jr,6-5,211,2/23/1998,"Stockholm, Sweden",Defensemen,77,2016.0,MTL,7.0,Des Moines,USHL,Lake-Superior
4,NaN,20,"Hindman, Grant",Fr,6-0,200,5/16/2002,"Oakland, Mich.",Defensemen,72,NaN,NaN,NaN,Youngstown,USHL,Lake-Superior


## Create dictionary of Team Primary Names to Abbreviations

- Needed to Add IVY teams becuase of low amount of games. will have to do for harvard, yale, ect next week


In [3]:
## Create dataframe from SQL query
df = pd.read_sql_query("SELECT * FROM advanced_metrics", conn)

# Function to count the occurrences of primary team names for unmatched abbreviations
def count_primary_names_for_abbreviation(abbreviation):
    filtered_rows = df[df['Team'] == abbreviation]
    team_counts = {}
    
    for _, row in filtered_rows.iterrows():
        teams = row['Game_ID'].split('-')[-2:]
        for team in teams:
            if team not in team_counts:
                team_counts[team] = 0
            team_counts[team] += 1
            
    return team_counts


# Attempt to match abbreviations to primary names based on substrings and common naming conventions
matched_dict = {}
unmatched_abbreviations = []

for abbreviation in df['Team'].unique():



# Match the abbreviation to the primary team name with the highest occurrence

    team_counts = count_primary_names_for_abbreviation(abbreviation)
    # Get the team with the highest count
    matched_team = max(team_counts, key=team_counts.get)
    matched_dict[abbreviation] = matched_team

# matched_dict

# Manually fix the unmatched abbreviations - IVY League Teams with no of very few games throw a wrench in the above method
# Brown: Brown
# Cornell: Cornell

# Make those substitutions
matched_dict['Brown'] = 'Brown'
matched_dict['Cornell'] = 'Cornell'
# yale
matched_dict['Yale'] = 'Yale'
# princeton
matched_dict['Princeton'] = 'Princeton'

# harvard
matched_dict['Harvard'] = 'Harvard'
# columbia
matched_dict['Columbia'] = 'Columbia'

# dartmouth
matched_dict['Dartmouth'] = 'Dartmouth'

# penn
matched_dict['Penn'] = 'Penn'

# BC
matched_dict['BC'] = 'Boston College'



# matched_dict

## Clean and Transform Advanced Metrics
- add, Team and Home-Away columns, combine the two tables into a single table

In [4]:
df.columns

Index(['Team', 'Player', 'Bl', 'Mi', 'SV', 'G', 'TSA', 'Bl_1', 'Mi_1', 'SV_1',
       'G_1', 'TSA_1', 'Bl_2', 'Mi_2', 'SV_2', 'G_2', 'TSA_2', 'Bl_3', 'Mi_3',
       'SV_3', 'G_3', 'TSA_3', 'BLKs', 'FO', 'Game_ID'],
      dtype='object')

In [5]:
## NEW Handling of Advanced Stats
# Create dataframe from SQL query
df = pd.read_sql_query("SELECT * FROM advanced_metrics", conn)

# Rename columns
new_names = ['Team', 'Player', 'TOTAL_Block', 'TOTAL_Miss', 'TOTAL_Saved', 'TOTAL_Goals', 'TOTAL_Total_Shots',
                'EVEN_Block', 'EVEN_Miss', 'EVEN_Saved', 'EVEN_Goals', 'EVEN_Total_Shots',
                'PP_Block', 'PP_Miss', 'PP_Saved', 'PP_Goals', 'PP_Total_Shots',
                'CLOSE_Block', 'CLOSE_Miss', 'CLOSE_Saved', 'CLOSE_Goals', 'CLOSE_Total_Shots',

                'D_Blocks', 'Faceoffs', 'Game_ID']

df.columns = new_names

# # Apply the matched_dict to the Team column
df['Team'] = df['Team'].apply(lambda x: matched_dict[x])

## Fill all NaN values with 0
df = df.fillna(0)

# Display the dataframe
df.head()

# Save back to the database
df.to_sql('advanced_metrics', conn, if_exists='replace', index=False)

43060

In [6]:
### OBSOLETE SINCE NOV 2 BECAUSE OF UPDATE TO SCRAPING BOOK
# ### NEW Simple - advanced metrics 1 and 2 now have a team column (still contains the secondary team name so will want to address that as well
# ## Combine the two tables into advanced_metrics_combined

# # Get entire first table of advanced metrics
# adv_met_1 = pd.read_sql_query("SELECT * FROM advanced_metrics_team1", conn)


# # Get entire second table of advanced metrics
# adv_met_2 = pd.read_sql_query("SELECT * FROM advanced_metrics_team2", conn)


# # Combine the two tables
# adv_met_combined = pd.concat([adv_met_1, adv_met_2], axis=0)

# # Apply the matched_dict to the Team column
# adv_met_combined['Team'] = adv_met_combined['Team'].apply(lambda x: matched_dict[x])


# # Fill NaNs with 0
# adv_met_combined = adv_met_combined.fillna(0)

# # Add to database
# adv_met_combined.to_sql('advanced_metrics_combined', conn, if_exists='replace', index=False)

# # View Sample and output CSV
# adv_met_combined.head()

# # adv_met_combined.to_csv('../TEMP/NEWNEW advanced_metrics_combined.csv', index=False)

## Add Home and Away Columns to game_details table

In [7]:
# Step 1: Read the game_details table into a DataFrame
df_game_details = pd.read_sql("SELECT * FROM game_details", conn)

# Step 2: Create new columns for Home and Away Teams by parsing Game_ID
df_game_details['Away_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[3])
df_game_details['Home_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[4])

# Step 3: Write this updated DataFrame back to the game_details table
df_game_details.to_sql('game_details', conn, if_exists='replace', index=False)


1083

## Clean up The Column Names and extra header rows in the Player Stats table

In [8]:
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

# view sample
player_stats_df.head()

,Team,Player,G,A,Pt.,+/-,Sh,PIM,FOW,FOL,FO%,Game_ID
0,Army,Army,G,A,Pt.,+/-,Sh,PIM,NaN,NaN,NaN,2021-10-02-Army-Providence
1,Army,Joey Baez,0,0,0,-1,2,0,NaN,NaN,NaN,2021-10-02-Army-Providence
2,Army,Michael Sacco,0,0,0,-3,1,0,NaN,NaN,NaN,2021-10-02-Army-Providence
3,Army,Thomas Farrell,0,0,0,-2,2,0,NaN,NaN,NaN,2021-10-02-Army-Providence
4,Army,Justin Evenson,0,0,0,0,0,0,NaN,NaN,NaN,2021-10-02-Army-Providence


In [9]:
# Define a dictionary for column renaming
column_renames = {
    'Pt.': 'Pts',
    '+/-': 'plus_minus'
}

# Rename columns based on the dictionary
player_stats_df.rename(columns=column_renames, inplace=True)


# Drop rows where Team name is in the Player column
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# Print the length of the dataframe
len(player_stats_df)


43278

In [10]:
## Change the Column names to be easy to work with
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

if 'Pt.' in player_stats_df.columns:
    player_stats_df.rename(columns={'Pt.': 'Pts'}, inplace=True)
else:
    print("Column 'Pt.' not found.")

if '+/-' in player_stats_df.columns:
    player_stats_df.rename(columns={'+/-': 'plus_minus'}, inplace=True)
else:
    print("Column '+/-' not found.")

print(len(player_stats_df))

# Drop rows if Team name is in the player column
# If ['Team'] is the same as ['Player'] then drop that row
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# add the dataframe back to the database
player_stats_df.to_sql('player_stats', conn, if_exists='replace', index=False)

# print(len(player_stats_df))
#################################
# player_stats_df.head()

45444


43278

# Check for other subsitu

In [11]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame
df_linescores = pd.read_sql("SELECT * FROM linescore", conn)

# Apply the dictionary to the Team column
df_linescores['Team'] = df_linescores['Team'].apply(lambda x: matched_dict[x])

df_linescores.head()

,Team,goals1,goals2,goals3,goalsT,shots1,shots2,shots3,shotsT,Pen,...,FOW,FOL,FOW%,goals4,goals5,goals6,shots4,shots5,shots6,Game_ID
0,Army,0,0,0,0,8,7,10,25,3,...,23,39,37.096774,0,0,0,0,0,0,2021-10-02-Army-Providence
1,Providence,2,2,3,7,11,14,15,40,3,...,39,23,62.903226,0,0,0,0,0,0,2021-10-02-Army-Providence
2,Sacred Heart,1,2,0,3,7,14,13,34,1,...,27,37,42.187500,0,0,0,0,0,0,2021-10-02-Sacred Heart-Connecticut
3,Connecticut,2,1,3,6,9,9,10,28,4,...,37,27,57.812500,0,0,0,0,0,0,2021-10-02-Sacred Heart-Connecticut
4,Miami,2,4,1,7,7,11,4,22,4,...,25,25,50.000000,0,0,0,0,0,0,2021-10-02-Miami-Ferris State


# Penalty Table & Scoring Summary

In [12]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame

df_penalty = pd.read_sql("SELECT * FROM penalty_summary", conn)

# Apply the dictionary to the Team column
#$ Skip if not found

df_penalty['Team'] = df_penalty['Team'].apply(lambda x: matched_dict[x])

    

# Apply same method to scorring_summary:
df_scoring = pd.read_sql("SELECT * FROM scoring_summary", conn)
df_scoring['Team'] = df_scoring['Team'].apply(lambda x: matched_dict[x])

# Add Home and Away Team columns to scoring_summary
df_scoring['Away_Team'] = df_scoring['Game_ID'].apply(lambda x: x.split('-')[3])
df_scoring['Home_Team'] = df_scoring['Game_ID'].apply(lambda x: x.split('-')[4])


## Add each table back to database
# Write the updated linescores DataFrame back to the linescore table
df_linescores.to_sql('linescore', conn, if_exists='replace', index=False)

# Write the updated penalty DataFrame back to the penalty_summary table
df_penalty.to_sql('penalty_summary', conn, if_exists='replace', index=False)

# Write the updated scoring DataFrame back to the scoring_summary table
df_scoring.to_sql('scoring_summary', conn, if_exists='replace', index=False)

6125

## CREATE A NEW TABLE WITH AGGRIGATED PLAYER STATS YEAR TO DATE

In [13]:
# Use player_stats_df from here on, instead of running another SQL query.
df_player_stats = player_stats_df.copy()


# Clean up the name format in player_stats for easier matching
# Replace the non-breaking space with a regular space
df_player_stats['Clean_Player'] = df_player_stats['Player'].apply(lambda x: x.replace('\xa0', ' '))

# Remove rows where Player is the team name (e.g., "Michigan State")
df_player_stats_cleaned = df_player_stats[df_player_stats['Player'] != df_player_stats['Team']]

# Convert relevant columns to integers for correct aggregation
cols_to_convert = ['G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM']
for col in cols_to_convert:
    df_player_stats_cleaned[col] = pd.to_numeric(df_player_stats_cleaned[col], errors='coerce')

# Aggregate the data for year-to-date stats
# Add a column for counting the number of games each player has played
agg_player_stats_corrected_with_games = df_player_stats_cleaned.groupby(['Clean_Player', 'Team']).agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'PIM': 'sum',
    'Game_ID': 'count'  # Counting the number of unique Game_IDs for each player
}).reset_index()

# Rename the Game_ID column to Games_Played
agg_player_stats_corrected_with_games.rename(columns={'Game_ID': 'Games_Played'}, inplace=True)

# Save the updated aggregated data back to the database, replacing the existing table
agg_player_stats_corrected_with_games.to_sql('player_stats_ytd', conn, if_exists='replace', index=False)

# Verify by loading some sample data from the updated table
sample_updated_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd LIMIT 5;", conn)
sample_updated_ytd


,Clean_Player,Team,G,A,Pts,plus_minus,Sh,PIM,Games_Played
0,A.J. Hodges,Michigan State,0,4,4,-8,45,4,34
1,AJ Drobot,Maine,2,1,3,-3,26,6,23
2,AJ Vanderbeck,Northern Michigan,24,19,43,8,161,18,36
3,Aaron Bohlinger,Massachusetts,2,11,13,3,29,4,28
4,Aaron Grounds,American Int'l,1,1,2,3,11,4,15


## Add the Roster data from the CSVs to the Database

In [14]:
################## SET THE ROSTER DATAFRAME TO THE CORRECT YEAR ####################
## MATCH THE DATAFRAME NAMES
df_master_roster = roster_df.copy()

## Season Year Value
season_year = season_year_setting

# Clean up the name formats for joining
# Master roster: Convert "Last Name, First Name" to "First Name Last Name"
# df_master_roster['Clean_Name'] = df_master_roster['Player'].apply(lambda x: ' '.join(reversed(x.split(', '))))

# Rename Player to Clean_Name
df_master_roster.rename(columns={'Player': 'Clean_Name'}, inplace=True)
# Rename School to Team
df_master_roster.rename(columns={'School': 'Team'}, inplace=True)

# Clean up the Team column, remove '-' and replace with ' '
# df_master_roster['School'] = df_master_roster['Team'].apply(lambda x: x.replace('-', ' '))

## If there are an period in the column names, remove them
df_master_roster.columns = df_master_roster.columns.str.replace('.', '')



In [15]:
### Finally add the roster to the database as it's own table

df_master_roster['SeasonYear'] = season_year

# Save the roster data as a new table in the database
roster_table_name = 'master_roster'
df_master_roster.to_sql(roster_table_name, conn, if_exists='replace', index=False)
############################################################

# Verify by listing all the tables in the database again
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(tables_query).fetchall()
table_names_updated = [table[0] for table in tables]
table_names_updated


['goalie_stats',
 'line_chart',
 'advanced_metrics',
 'game_details',
 'player_stats',
 'linescore',
 'penalty_summary',
 'scoring_summary',
 'player_stats_ytd',
 'master_roster']

## Save a backup of the transformed database and proceed to adding the roster info


In [16]:
# # # Output the current state of the database to a new SQLite file in the temp folder
# # db_cleaned_path ='../TEMP/new_cleaned_db.db'

# ## Save the database to a new file
# conn_cleaned = sqlite3.connect(db_cleaned_path)





